In [ ]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
from textblob import TextBlob  # For sentiment analysis
from transformers import pipeline, AutoTokenizer  # HuggingFace transformers for BERT integration
from tabulate import tabulate  # For beautifully formatted console outputs

# Advanced Preprocessing Function
def preprocess_text(df, column):
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace(r'[^\w\s]', '', regex=True)
    df['text_length'] = df[column].apply(len)
    df['sentiment'] = df[column].apply(lambda x: TextBlob(x).sentiment.polarity)
    return df

# Load datasets
true_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')
manual_testing_data = pd.read_csv('manual_testing.csv')

# Add labels
true_data['label'] = 1  # 1 = True
fake_data['label'] = 0  # 0 = Fake

# Combine datasets for training
data = pd.concat([true_data, fake_data], axis=0).reset_index(drop=True)

# Preprocess the data
data = preprocess_text(data, 'text')
manual_testing_data = preprocess_text(manual_testing_data, 'text')

# Text Representation
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(data['text'])
y_train = data['label']

# Model Training (Random Forest)
print("\n🚀 **Training the Random Forest model on True and Fake datasets...**")
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_tfidf, y_train)
print("✅ Random Forest model training complete!")

# Load BERT Model
bert_classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
bert_tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Truncate Text for BERT
def truncate_text(text, max_length=512):
    tokens = bert_tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        add_special_tokens=True  # Includes [CLS] and [SEP]
    )
    return bert_tokenizer.decode(tokens['input_ids'], skip_special_tokens=True)

# Apply truncation
manual_testing_data['truncated_text'] = manual_testing_data['text'].apply(lambda x: truncate_text(x, max_length=512))

# Manual Testing
print("\n📊 **Evaluating manual testing dataset...**")
manual_tfidf = tfidf.transform(manual_testing_data['text'])
manual_predictions = rf_model.predict(manual_tfidf)

# Add Predictions to Manual Testing Data
manual_testing_data['RF_Predicted Label'] = manual_predictions
manual_testing_data['BERT_Predicted Sentiment'] = manual_testing_data['truncated_text'].apply(
    lambda x: bert_classifier(x)[0]['label']
)

# Display Predictions in Tabular Form
print("\n🔍 **Manual Testing Results:**")
results_table = tabulate(
    manual_testing_data[['text', 'RF_Predicted Label', 'BERT_Predicted Sentiment']],
    headers='keys',
    tablefmt='grid',
    maxcolwidths=[50, 20, 20]
)
print(results_table)

# Check for Ground Truth in Manual Testing Data
if 'label' in manual_testing_data.columns:
    manual_testing_data['label'] = manual_testing_data['label'].map({0: 'Fake', 1: 'True'})
    y_manual_test = manual_testing_data['label'].map({'Fake': 0, 'True': 1})
    accuracy = accuracy_score(y_manual_test, manual_predictions)
    print(f"\n✨ **Evaluation on Manual Testing Data:**\nAccuracy: {accuracy:.2%}")
    print("\n🔔 **Classification Report:**")
    print(classification_report(y_manual_test, manual_predictions, target_names=['Fake', 'True']))

    # AUC-ROC Curve
    rf_probs = rf_model.predict_proba(manual_tfidf)[:, 1]
    roc_auc = roc_auc_score(y_manual_test, rf_probs)
    print(f"AUC-ROC: {roc_auc:.2f}")

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_manual_test, manual_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# Visualizing Prediction Summary
summary = manual_testing_data['RF_Predicted Label'].value_counts()
summary_percentage = (summary / summary.sum()) * 100
plt.figure(figsize=(8, 6))
sns.barplot(x=summary.index, y=summary.values, palette='viridis')
plt.title('Prediction Summary: True vs. Fake Articles')
plt.xlabel('Article Type')
plt.ylabel('Number of Articles')
plt.show()
